In [ ]:
# Kerasを用いて画像分類モデル（特にAlexNetのような構造）を構築し、訓練する

# ライブラリのインポート
import os
import numpy as np
import matplotlib.pyplot as plt
from keras.models import Sequential, Model
from keras.layers import Input, Activation, Dropout, Flatten, Dense, Conv2D, MaxPooling2D, BatchNormalization
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator
from keras.initializers import TruncatedNormal, Constant
from keras.utils import np_utils
import glob
from PIL import Image



# データセットの読み込みと前処理
image_size = 28
X = []
Y = []

# 画像データの読み込みと前処理
folder = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9']
for index, name in enumerate(folder):
    dir = './' + name
    files = glob.glob(dir + '/*.png')
    for file in files:
        image = Image.open(file)
        image = image.resize((image_size, image_size))
        data = np.asarray(image)
        X.append(data)
        Y.append(index)

X = np.array(X)
Y = np.array(Y)
X = X.astype('float32') / 255.0
Y = np_utils.to_categorical(Y, 10)

# モデルの構築
def AlexNet(image_size):
    model = Sequential()

    # 畳み込み層とプーリング層
    model.add(Conv2D(96, (11, 11), strides=(4,4), activation='relu', input_shape=(image_size, image_size, 3)))
    model.add(MaxPooling2D((3, 3), strides=(2, 2)))
    model.add(BatchNormalization())

    model.add(Conv2D(256, (5, 5), activation='relu', padding='same'))
    model.add(MaxPooling2D((3, 3), strides=(2, 2)))
    model.add(BatchNormalization())

    model.add(Conv2D(384, (3, 3), activation='relu', padding='same'))
    model.add(Conv2D(384, (3, 3), activation='relu', padding='same'))
    model.add(Conv2D(256, (3, 3), activation='relu', padding='same'))
    model.add(MaxPooling2D((3, 3), strides=(2, 2)))
    model.add(BatchNormalization())

    # 全結合層
    model.add(Flatten())
    model.add(Dense(4096, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(4096, activation='relu'))
    model.add(Dropout(0.5))

    # 出力層
    model.add(Dense(10, activation='softmax'))

    # モデルのコンパイル
    model.compile(optimizer=Adam(lr=0.001), loss='categorical_crossentropy', metrics=['accuracy'])
    
    return model

model = AlexNet(image_size)
model.summary()


# 訓練
# ImageDataGeneratorのインスタンス化
train_datagen = ImageDataGenerator(rescale=1.0 / 255, rotation_range=30, width_shift_range=0.2, height_shift_range=0.2, horizontal_flip=True, zoom_range=0.2, shear_range=0.2)
test_datagen = ImageDataGenerator(rescale=1.0 / 255)

# ディレクトリからのデータの読み込みとジェネレータの設定
train_generator = train_datagen.flow_from_directory(train_data_dir, target_size=(image_size, image_size), batch_size=batch_size, class_mode='binary')
validation_generator = test_datagen.flow_from_directory(validation_data_dir, target_size=(image_size, image_size), batch_size=batch_size, class_mode='binary')

# モデルの訓練
history = model.fit_generator(train_generator, steps_per_epoch=num_train // batch_size, epochs=epochs, validation_data=validation_generator, validation_steps=num_validation // batch_size)


# 損失の履歴をプロット
plt.plot(history.history['loss'], label='Training loss')
plt.plot(history.history['val_loss'], label='Validation loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

# 精度の履歴をプロット
plt.plot(history.history['accuracy'], label='Training accuracy')
plt.plot(history.history['val_accuracy'], label='Validation accuracy')
plt.title('Training and Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()






